# Chemical equilibrium with custom constraints

Reaktoro already implements *the list of chemical equilibrium constraints* that can be checked in API of [EquilibriumSpecs](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSpecs.html) class. If available constraints do not suffice, the **customized constraints** can be defined and used.

Before we proceed, we need to understand how equilibrium constraints are formulated in Reaktoro and used by its equilibrium solver. Let's assume the **enthalpy of the system** must be a constraint to $H_{\mathrm{desired}}$. At each iteration of equilibrium calculation, the equilibrium solver generates the chemical state that honours all imposed equilibrium constraints and mass/charge conservation conditions with the enthalpy of the system $H_{\mathrm{current}}$. At some point, the algorithm will converge to a chemical state in which the *equilibrium residual constraint* satisfies, i.e.,

$$|H_{\mathrm{current}}-H_{\mathrm{desired}}|<\epsilon_{\mathrm{tolerance}}$$

where $\epsilon_{\mathrm{tolerance}}$ is a small positive tolerance value (e.g., 10<sup>-6</sup>).

Therefore, constraints in Reaktoro can be defined by writing **residual expressions** similar but not limited to the one above (e.g., involving multiple properties).

## Creating custom volume and internal energy constraints

In a [previous example](equilibrium-with-fixed-volume-internal-energy.ipynb), we modeled the combustion of CH<sub>4</sub> in a rigid and adiabatic chamber. We repeat this experiment here providing **customized volume and internal energy constraints**.

Below, we recall the chemical system and corresponding chemical state and calculate the chemical properties of this system to fetch the initial volume and internal energy in the variables `V0` and `U0` for later use:

In [ ]:
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 O2 CO2 CO H2O H2")

system = ChemicalSystem(db, gases)

state = ChemicalState(system)
state.temperature(25.0, "celsius")
state.pressure(1.0, "bar")
state.set("CH4", 0.75, "mol")
state.set("O2",  0.25, "mol")
state.scaleVolume(10.0, "cm3")

props0 = ChemicalProps(state)

V0 = props0.volume()
U0 = props0.internalEnergy()

Next, let's define custom constraints (see more detailed comments below):

In [ ]:
# Define the equilibrium specifications
specs = EquilibriumSpecs(system)

# Define the indices of newly added constraints
idxV = specs.addInput("V")  # add "V" as the symbol for a new input condition to the equilibrium problem
idxU = specs.addInput("U")  # add "U" as the symbol for a new input condition to the equilibrium problem

# Define volume constraint
volumeConstraint = ConstraintEquation()
volumeConstraint.id = "VolumeConstraint"  # give some identification name to the constraint (it's up to you how you call this)
volumeConstraint.fn = lambda props, w: props.volume() - w[idxV]  # the residual function defining V(current) - V(desired)

# Define internal energy constraint
internalEnergyConstraint = ConstraintEquation()
internalEnergyConstraint.id = "InternalEnergyConstraint"  # give some identification name to the constraint (it's up to you how you call this)
internalEnergyConstraint.fn = lambda props, w: props.internalEnergy() - w[idxU]  # the residual function defining U(current) - U(desired)

# Add new constraints to equilibrium specs
specs.addConstraint(volumeConstraint)
specs.addConstraint(internalEnergyConstraint)

# Define the equilibrium solver using specs defined above
solver = EquilibriumSolver(specs)


In addition to the code comments above, it's worth commenting further that:

* `idxV` and `idxU` are the indices of the newly added constraints to the equilibrium problem (namely `V` and `U`)
* `lambda props, w: props.volume() - w[idxV]` is a [lambda-function](https://www.w3schools.com/python/python_lambda.asp) defining custom *residual expression for the volume constraint*: $$V_\mathrm{current} - V_\mathrm{desired}$$
* `lambda props, w: props.internalEnergy() - w[idxV]` is a lambda function defining custom *residual expression for the internal energy constraint*
* `props` is the [ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html) object containing the current chemical properties of the system, i.e., *current volume and internal energy*
* `w` is the array with the input values introduced in the equilibrium problem where we store the *desired volume and internal energy*
* `w[idxV]` and `w[idxU]` are the desired values of introduced inputs `V` and `U`

It's now time to create our [EquilibriumConditions](https://reaktoro.org/api/classReaktoro_1_1EquilibriumConditions.html) object, where specify the values for the inputs `V` and `U`:

In [ ]:
conditions = EquilibriumConditions(specs)
conditions.set("V", V0)  # use the constraint with the symbol "V" introduced above
conditions.set("U", U0)  # use the constraint with the symbol "V" introduced above

conditions.setLowerBoundTemperature(25.0, "celsius")
conditions.setUpperBoundTemperature(2000.0, "celsius")

conditions.setLowerBoundPressure(1.0, "bar")
conditions.setUpperBoundPressure(1000.0, "bar")

Solve the chemical equilibrium problem that models the combustion of CH<sub>4</sub> in a rigid and adiabatic chamber:

In [ ]:
solver.solve(state, conditions)

print("FINAL STATE")
print(state)

**TASK 1**: 1) check if this computed equilibrium state is identical to that found in this [tutorial](equilibrium-with-fixed-volume-internal-energy.ipynb) 2) verify that equilibrium state has volume and internal energy equal to `V0` and `U0`, respectively.

In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #

## Creating custom pH constraint

**Problem:** how to *fix the pH* of an aqueous solution by *titrating it with another substance*?

Assume the chemical system with an aqueous solution saturated with mineral calcite (CaCO<sub>3</sub>) at 30 °C and 5 atm. How much CO<sub>2</sub> must be titrated into the system to obtain pH 7? This problem can be solved by formulating a chemical equilibrium calculation in which **temperature**, **pressure**, and **pH** are constrained and **the system is open to the mass transfer of CO<sub>2</sub>**.

The definition of a chemical system includes the following phases:

* an aqueous phase (with aqueous species that can be formed from elements H, O, Na, Cl, C, and Ca),
* a gaseous phase (with only CO<sub>2</sub>(g) and H<sub>2</sub>O(g) species), and
* a solid phase (representing the mineral calcite):

In [ ]:
from reaktoro import *

db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase(speciate("H O Na Cl C Ca"))
solution.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2")
))

gases = GaseousPhase("CO2(g) H2O(g)")
gases.setActivityModel(ActivityModelPengRobinson())

calcite = MineralPhase("Calcite")

system = ChemicalSystem(db, solution, gases, calcite)

We define a helper function that computes pH for given chemical properties of a system:

In [ ]:
# Fetch H+ index to avoid search in the system at each call
idxH = system.species().index("H+")
# Calculate pH for a given chemical properties
def pH(props: ChemicalProps):
    return -props.speciesActivityLg(idxH)  # this results in pH = -log10(a[H+])

We recap that we aim to perform a chemical equilibrium calculation *with constraints* of

* temperature,
* pressure, and
* pH,

allowing the chemical system to be open to CO<sub>2</sub>.

**TASK 2**: using [EquilibriumSpecs](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSpecs.html) class, 1) define custom pH constraint and initialize all the constraints of current problem.

In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #

An *initial chemical state* for the system is represented with a 1 molal NaCl aqueous solution mixed with a sufficient amount of calcite to saturate the fluid at 25 °C and 1 atm:

In [ ]:
state = ChemicalState(system)
state.temperature(25.0, "celsius")
state.pressure(1.0, "atm")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("Calcite", 10.0, "mol")  # plenty of Calcite to ensure saturation levels!

Finally, we set the conditions of this system at the chemical equilibrium state of interest (using [EquilibriumConditions](https://reaktoro.org/api/classReaktoro_1_1EquilibriumConditions.html)) and compute this state:

In [ ]:
conditions = EquilibriumConditions(specs)
conditions.temperature(30.0, "celsius")
conditions.pressure(5.0, "atm")
conditions.set("pH", 7.0)  # remember the symbol "pH" introduced before? you're using it here!

solver = EquilibriumSolver(specs)
solver.solve(state, conditions)

aprops = AqueousProps(state)
print(aprops)

The aqueous properties displayed above should show that the prescribed temperature, pressure, and pH values are satisfied. 

**TASK 3**: check how much CO<sub>2</sub> was transferred to the system for this pH value to be attained.

In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #